In [1]:
from pymongo import MongoClient
from pprint import pprint
import random
import gender_guesser.detector as gender
import re

In [2]:
## Mongo
client = MongoClient('mongodb://localhost:27017')
db=client.dat
collection = db.crossref

## Gender estimation
detector = gender.Detector(case_sensitive=False)



In [ ]:
import logging

def get_gender(detector_instance, author):
    """ Analyzes author dict to return one of several results relevant to project.
    """
    if not isinstance(author, dict):
        return 'unknown-data_failure'

    try:
        if 'given' not in author or len(author['given']) == 0:
            return 'unknown-no_first_name'
        name = author['given'].split(' ')[0] # Take only first word

        if len(name) == 1:
            return 'unknown-single_character_name'

        name_sans_initials = re.sub(r'([\w\W]{1}[.]{1})', '', name)
        if not len(name_sans_initials):
            return 'unknown-initials_only'

        gender_estimate = d.get_gender(name)
        if gender_estimate == 'unknown':
            return 'unknown-estimation_failure'

        return gender_estimate

    except Exception, e:
        logging.info('Was working on author string: ', str(author))
        logging.exception(e)
        return 'unknown-code_failure'

def set_gender(collection, record_id, genders):
    collection.update(
        {"_id": record_id},
        {"$set":
            {"author-genders": genders}
        }
    )

def get_cursor():
    """ Get cursor pointing at records without the field 'author-genders'.
        Return True and a cursor if cursor has at least 1 records. Otherwise
        returns False and None. This is to avoid using .count() on big cursor.
    """
    c = collection.find({
        'author-genders' : { '$exists' : False },
        'author' : {'$exists' : True}
        })

    counter = 0
    for record in c:
        c.rewind()
        # If you're in this loop there's something in the cursor
        return {'empty' : False, 'cursor' : c}
    return {'empty' : True, 'cursor' : c}


# Get cursor, loop through it, estimate genders + write estimates to mongo
results_exist = True
finished = 0

while results_exist:
    try:
        cursor = get_cursor()
        if not cursor['empty']:
            for publication in cursor['cursor']:
                genders = []
                for auth in publication['author']:
                    gender = get_gender(detector, auth)
                    genders.append(gender)

                set_gender(collection, publication['_id'], genders)
                finished += 1
                if finished % 10 == 0:
                    print 'Finished {} records so far.'.format(finished)

        else:
            results_exist = False

    except Exception, e:
        print e


In [4]:
# Count number of currently-inserted records. 
# This is faster than calling .count() on cursor if only a few results exist.
c = collection.find({
        'author-genders' : { '$exists' : False },
        'author' : {'$exists' : True}
        })
total = 0
for item in c:
    total += 1
    if total % 100000 == 0:
        print total
'finished, total is', total

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

KeyboardInterrupt: 